# How to evaluate a RAG application

This example uses [Langchain](https://www.langchain.com) and [Giskard](https://github.com/Giskard-AI/giskard) to evaluate the quality of a RAG application.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"

## Scrape the Website and Split the Content

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

loader = WebBaseLoader("https://www.ml.school/")
documents = loader.load_and_split(text_splitter)
documents

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://www.ml.school/', 'title': "Building Machine Learning Systems That Don't Suck", 'description': "A live, interactive program that'll help you build production-ready machine learning systems from the ground up.", 'language': 'en'}, page_content='Building Machine Learning Systems That Don\'t Suck"This is the best machine learning course I\'ve done. Worth every cent."Jose Reyes, AI/ML at Cevo AustraliaBuilding Machine Learning Systems That Don\'t SuckA live, interactive program that\'ll help you build production-ready machine learning systems from the ground up.Next cohort:\xa0November 4 - 21, 2024Check the schedule for more details about upcoming cohorts.I want to join!Sign inLearn how to design, build, deploy, and scale machine learning systems to solve real-world problems.I\'ll lose my mind if I see another book or course teaching people the same basic ideas for the hundredth time. Most people are stuck in beginner mode, and finding help to solve re

## Load the Content in a Vector Store

In [3]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    documents, embedding=OpenAIEmbeddings()
)

/Users/derinberktay/Desktop/LLM/testing video/.venv/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


## Create a Knowledge Base

Let's start by loading the content in a pandas DataFrame.

In [4]:
import pandas as pd

df = pd.DataFrame([d.page_content for d in documents], columns=["text"])
df.head(10)

,text
0,Building Machine Learning Systems That Don't S...
1,program will help you unlearn what you think m...
2,only pay once to join. There are no monthly fe...
3,that make systems work.You are ready to put in...
4,"testing in production, among many others.You'l..."
5,"Wednesdays, we'll host office hours when you c..."
6,as you'd like. No restrictions.Enjoy 18 hours ...
7,to determine how much data you need.The proble...
8,"it with complete confidence.""Juan OlanoMachine..."
9,"learning, beginners will find the sessions go ..."


We can now create a Knowledge Base using the DataFrame we created before.

In [5]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(df)

/Users/derinberktay/Desktop/LLM/testing video/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Generate the Test Set

In [8]:
from giskard.rag import generate_testset
testset = generate_testset(
    knowledge_base,
    num_questions=60,
    agent_description="A chatbot answering questions about the Machine Learning School Website",
)

Generating questions: 100%|██████████| 60/60 [07:51<00:00,  7.87s/it]


Let's display a few samples from the test set.

In [9]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")


Question 1: Who holds the copyright for the content?
Reference answer: The content is copyrighted by Tideily LLC.
Reference context:
Document 10: then, thousands of students have graduated, and I can't wait to meet you in class.Copyright © 2024 Tideily LLCAll rights reserved.
******************

Question 2: What are the benefits and features of the machine learning program?
Reference answer: The machine learning program offers 18 hours of live, interactive sessions and 10 hours of step-by-step coding instructions. Participants will also get to work on a final project, complete 100 coding assignments, and gain access to the source code of a working production system. Other benefits include access to a private community for collaboration, direct access to the instructor, lifetime access to all past and future cohorts, and a program certificate upon completion. Participants only need to pay once to join the program, with no monthly or annual fees.
Reference context:
Document 1: program wi

Let's now save the test set to a file:

In [10]:
testset.save("test-set.jsonl")

## Prepare the Prompt Template

In [11]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



## Create the RAG Chain

Create a retriever from the Vector Store that will allow us to get the top similar documents to a given question.

In [12]:
retriever = vectorstore.as_retriever()
retriever.get_relevant_documents("What is the Machine Learning School?")

/var/folders/dv/x_yhvm8x1vj_7019b6vm83pr0000gn/T/ipykernel_54726/814621628.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("What is the Machine Learning School?")


[Document(metadata={'source': 'https://www.ml.school/', 'title': "Building Machine Learning Systems That Don't Suck", 'description': "A live, interactive program that'll help you build production-ready machine learning systems from the ground up.", 'language': 'en'}, page_content="program will help you unlearn what you think machine learning is. It's a practical, hands-on class where you'll learn from years of experience and real-world examples.When you join, you get lifetime access to the following:18 hours of live, interactive sessions. We'll use this time to discuss the first principles behind building machine learning systems.10 hours of step-by-step coding instructions. These practical sessions will show you how to build an end-to-end system from scratch.A final project where you'll build a complete solution and receive direct feedback on your work.100 coding assignments and practice questions.The entire source code of a working production system. It's yours. You can change and us

We can now create our chain.

In [13]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

Let's make sure the chain works by testing it with a simple question.

In [14]:
chain.invoke({"question": "What is the Machine Learning School?"})

'The Machine Learning School is an online program that offers live, interactive sessions and step-by-step coding instructions to help individuals build production-ready machine learning systems from scratch. It also includes a final project, coding assignments, access to a private community, direct access to instructors, and a program certificate upon completion.'

## Evaluating the Model on the Test Set

We need to create a function that invokes the chain with a specific question and returns the answer.

In [15]:
def answer_fn(question, history=None):
    return chain.invoke({"question": question})

We can now use the `evaluate()` function to evaluate the model on the test set. This function will compare the answers from the chain with the reference answers in the test set.

In [16]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

CorrectnessMetric evaluation: 100%|██████████| 60/60 [02:02<00:00,  2.05s/it]


Let now display the report.

Here are the five components of our RAG application:

* **Generator**: This is the LLM used in the chain to generate the answers.
* **Retriever**: This is the retriever that fetches relevant documents from the knowledge base according to a query.
* **Rewriter**: This is a component that rewrites the user query to make it more relevant to the knowledge base or to account for chat history.
* **Router**: This is a component that filters the query of the user based on his intentions.
* **Knowledge Base**: This is the set of documents given to the RAG to generate the answers.

In [17]:
display(report)

Loading BokehJS ...

In [18]:
report.to_html("report.html")

We can display the correctness results organized by question type.

In [19]:
report.correctness_by_question_type()

,correctness
question_type,
complex,0.8
conversational,0.8
distracting element,0.6
double,0.8
simple,0.6
situational,0.7


We can also display the specific failures.

In [20]:
report.get_failures()

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
73383afa-9b5c-42f5-b49e-cbc319ecb9aa,Who holds the copyright for the Machine Learni...,The copyright for the Machine Learning School ...,"Document 10: then, thousands of students have ...",[],"{'question_type': 'simple', 'seed_document_id'...",I don't know.,False,The agent failed to provide the correct inform...
28ba93af-06ef-4750-993e-43c537ff8162,Who holds the copyright for the Machine Learni...,The copyright for the Machine Learning School ...,"Document 10: then, thousands of students have ...",[],"{'question_type': 'simple', 'seed_document_id'...",I don't know.,False,The agent did not provide the correct informat...
33c886f6-d076-48c0-bc44-0b6cf65f704f,What are the benefits of joining the machine l...,The benefits of joining the machine learning p...,Document 6: as you'd like. No restrictions.Enj...,[],"{'question_type': 'simple', 'seed_document_id'...",The benefits of joining the machine learning p...,False,The agent's answer is partially correct but it...
fdf26660-66c8-4938-96cf-5a62bb202fc1,What are the topics covered in the second sess...,The second session of the course covers topics...,Document 7: to determine how much data you nee...,[],"{'question_type': 'simple', 'seed_document_id'...",The topics covered in the second session of th...,False,The agent's answer missed some topics such as ...
a6733e27-77c0-4cef-9f20-650f576a0fa4,Could you provide me with the name and backgro...,The instructor of the program is Santiago. He ...,"Document 9: learning, beginners will find the ...",[],"{'question_type': 'complex', 'seed_document_id...",The document does not provide specific informa...,False,The agent failed to provide the correct inform...
e8016d43-cc84-4cda-bf92-2940cb3c2142,Could you clarify the financial commitment req...,The payment model for the Machine Learning Sch...,"Document 5: Wednesdays, we'll host office hour...",[],"{'question_type': 'complex', 'seed_document_id...",The financial commitment required to participa...,False,The agent did not provide the specific cost of...
1f397a2c-70bb-4c69-bb8e-a0bfc6f518e6,What are the topics covered in Session 2 of th...,"Session 2 covers topics such as data cleaning,...",Document 7: to determine how much data you nee...,[],"{'question_type': 'distracting element', 'seed...",The topics covered in Session 2 that are most ...,False,The agent's answer missed some topics covered ...
64d1305d-8012-4561-9fcb-5fb36410af55,Can you elaborate on the benefits and features...,"The program offers 18 hours of live, interacti...",Document 2: only pay once to join. There are n...,[],"{'question_type': 'distracting element', 'seed...",I don't know.,False,The agent failed to provide any information ab...
3445983b-171d-440c-854a-02e4e820da43,Considering the intensity of the Machine Learn...,"The instructor of the program is Santiago, a m...","Document 9: learning, beginners will find the ...",[],"{'question_type': 'distracting element', 'seed...","Juan Olano, a Machine Learning Engineer, is th...",False,The instructor of the Machine Learning program...


## Creating a Test Suite

We can create a test suite and use it to compare different models.

Load the test set from disk.

In [21]:
from giskard.rag import QATestset

testset = QATestset.load("test-set.jsonl")

Create a Test Suite from the test set.

In [22]:
test_suite = testset.to_test_suite("Machine Learning School Test Suite")

We need a function that takes a DataFrame of questions, invokes the chain with each question, and returns the answers.

In [23]:
import giskard


def batch_prediction_fn(df: pd.DataFrame):
    return chain.batch([{"question": q} for q in df["question"].values])

We can now create a Giskard Model object to run our test suite.

In [24]:
giskard_model = giskard.Model(
    model=batch_prediction_fn,
    model_type="text_generation",
    name="Machine Learning School Question and Answer Model",
    description="This model answers questions about the Machine Learning School website.",
    feature_names=["question"], 
)

2024-10-07 16:49:35,719 pid:54726 MainThread giskard.models.automodel INFO     Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


Let's now run the test suite using the model we created before.

In [25]:
test_suite_results = test_suite.run(model=giskard_model)

2024-10-07 16:49:39,300 pid:54726 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2024-10-07 16:49:48,590 pid:54726 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (60, 5) executed in 0:00:09.302745
2024-10-07 16:51:28,216 pid:54726 MainThread root         ERROR    An error happened during test execution for test: TestsetCorrectnessTest
Traceback (most recent call last):
  File "/Users/derinberktay/Desktop/LLM/testing video/.venv/lib/python3.12/site-packages/giskard/core/suite.py", line 522, in run
    result = test_partial.giskard_test(**test_params).execute()
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/derinberktay/Desktop/LLM/testing video/.venv/lib/python3.12/site-packages/giskard/registry/giskard_test.py", line 195, in execute
    return configured_validate_arguments(self.test_fn)(*self.args, **self.kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^

We can display the results.

In [26]:
display(test_suite_results)

<TestSuiteResult (failed)>

## Integrating with Pytest

In [28]:
import ipytest

We can now integrate our test suite with Pytest.

In [30]:
%%ipytest

import pytest
from giskard.rag import QATestset
from giskard.testing.tests.llm import test_llm_correctness


@pytest.fixture
def dataset():
    testset = QATestset.load("test-set.jsonl")
    return testset.to_dataset()


@pytest.fixture
def model():
    return giskard_model


def test_chain(dataset, model):
    test_llm_correctness(model=model, dataset=dataset, threshold=0.5).assert_()

UsageError: Cell magic `%%ipytest` not found.
